In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [ ]:
page = requests.get('https://en.wikipedia.org/wiki/2018_FIFA_World_Cup_squads')
soup = BeautifulSoup(page.content,'html.parser')

#Find all tables in the page
tables = soup.findAll("table")

In [ ]:
team_details = soup.find_all('div', attrs={"class":"toc"})

In [ ]:
team_details[0].find_all('li',attrs={"class":"toclevel-2"})

In [ ]:
team_details[0].find_all('li',attrs={"class":"toclevel-2"})[0].contents[0].contents[2].contents[0]

In [ ]:
team_list = []

In [ ]:
for i in range(0,32):
    team_name = team_details[0].find_all('li',attrs={"class":"toclevel-2"})[i].contents[0].contents[2].contents[0]
    team_list.append(team_name)

In [ ]:
team_list

In [ ]:
players = tables[0].find_all('tr',attrs={"class":"nat-fs-player"})

In [ ]:
type(players)

In [ ]:
players[1].find_all('th')[0].contents[0].contents[0]

In [ ]:
players[0].find_all('td')[5].contents[2].contents[0]

In [ ]:
players[0].find_all('td')[5].contents[2].get('href')

In [ ]:
players[0].find_all('span',attrs={"class":"flagicon"})[0].contents[0].get('title')

In [ ]:
players[0].find_all('span',attrs={"class":"flagicon"})[0].contents[0].get('href')

In [ ]:
team_df = pd.DataFrame(columns=['Team'])

In [ ]:
team_df

In [ ]:
for i in range(0,len(team_list)):
    team_df.loc[i] = team_list[i]

In [ ]:
team_df

In [ ]:
full_df = pd.DataFrame(columns=['Team Name','Player Name','Club Name','Club Link'])

In [ ]:
temp_list=[]
player_list = []
team_list = []
for i in range(0,team_df['Team'].size):
    team_name = team_df['Team'][i];
    players = tables[i].find_all('tr',attrs={"class":"nat-fs-player"})
    for j in range(0,23):
        player_name = players[j].find_all('th')[0].contents[0].contents[0]
        club_name = players[j].find_all('td')[5].contents[2].contents[0]
        club_link = players[j].find_all('td')[5].contents[2].get('href')
        club_link = 'https://en.wikipedia.org' + club_link
#         print(club_link)
#         club_page = requests.get(club_link)
#         club_soup = BeautifulSoup(club_page.content,'html.parser')
#         info_table = club_soup.find('table',attrs={"class":"infobox vcard"})
#         info_table_rows = info_table.find_all('tr')
        """try:
            for i in range(0,10):
                if(info_table_rows[i].contents[0].contents[0]=='League'):
                    league_name=(info_table_rows[i].contents[1].contents[0].get('title'))
        except IndexError:
            print('Unavailable')
            break
        except:
            continue"""
        temp_list = [team_name,player_name,club_name,club_link]
        player_list.append(temp_list)
        print(temp_list)

In [ ]:
player_list

In [ ]:
player_df = pd.DataFrame.from_records(player_list,columns=['Nation','Name','Club','Link'])

In [ ]:
player_df.head(5)

In [ ]:
club_df = player_df[['Club','Link']]

In [ ]:
club_links = club_df.Link.unique()

In [ ]:
club_links

In [ ]:
club_list=[]

In [ ]:
temp_list = []
for i in range(0,len(club_links)):
    club_page = requests.get(club_links[i])
    club_soup = BeautifulSoup(club_page.content,'html.parser')
    info_table = club_soup.find('table',attrs={"class":"infobox vcard"})
    info_table_rows = info_table.find_all('tr')
    try:
        for j in range(0,12):
            if(info_table_rows[j].contents[0].contents[0]=='League'):
                league_name=(info_table_rows[j].contents[1].contents[0].get('title'))
    except IndexError:
        league_name = 'Unavailable'
        continue
    print(club_links[i],'--',league_name)
    temp_list = [club_links[i],league_name]
    club_list.append(temp_list)

In [ ]:
club_list

In [ ]:
club_link_df = pd.DataFrame.from_records(club_list,columns=['Link','League'])

In [ ]:
club_link_df[club_link_df['League']=='Unavailable']

In [ ]:
club_link_df_test = club_link_df

In [ ]:
player_df_test = player_df

In [ ]:
player_df_test.head(2)

In [ ]:
club_link_df_test.head(2)

In [122]:
player_df = player_df_test.merge(club_link_df_test,how='inner',left_on='Link',right_on='Link')

In [131]:
player_df = player_df.drop(['Link'],axis=1)

In [135]:
player_df = player_df.sort_values(by=['Nation','Name','Club'])

In [136]:
player_df

,Nation,Name,Club,League
346,Argentina,Cristian Ansaldi,Torino,Serie A
350,Argentina,Federico Fazio,Roma,Serie A
293,Argentina,Gabriel Mercado,Sevilla,La Liga
140,Argentina,Giovani Lo Celso,Paris Saint-Germain,Ligue 1
89,Argentina,Gonzalo Higuaín,Juventus,Serie A
104,Argentina,Lionel Messi,Barcelona,La Liga
192,Argentina,Lucas Biglia,Milan,Serie A
133,Argentina,Marcos Acuña,Sporting CP,Primeira Liga
233,Argentina,Marcos Rojo,Manchester United,Premier League
116,Argentina,Maximiliano Meza,Independiente,Argentine Primera División
